📒 Add paths and import required modules


In [ ]:
# Add root and utils to sys.path
import sys
import os

sys.path.append(os.path.abspath(".."))           # for model.py
sys.path.append(os.path.abspath("../utils"))     # for utils.py, logger.py

# Imports
import torch
import numpy as np
from model import ChordConditionedMelodyTransformer
from utils import pitch_to_midi
import matplotlib.pyplot as plt


📒 Define the model configuration
Make sure this matches your hparams.yaml.

In [ ]:
# Model configuration (should match training)
model_config = {
    "num_pitch": 50,
    "frame_per_bar": 16,
    "num_bars": 8,
    "chord_emb_size": 128,
    "pitch_emb_size": 256,
    "hidden_dim": 512,
    "key_dim": 512,
    "value_dim": 512,
    "num_layers": 8,
    "num_heads": 16,
    "input_dropout": 0.2,
    "layer_dropout": 0.2,
    "attention_dropout": 0.2
}


📒 Load the trained model from checkpoint


In [ ]:
# Load model checkpoint
checkpoint_path = "/home/cepatinog/smc-assignments/final_project/my_jazz_project/results/idx002/model/checkpoint_70.pth.tar"
checkpoint = torch.load(checkpoint_path, map_location="cpu")

model = ChordConditionedMelodyTransformer(**model_config)
model.load_state_dict(checkpoint["model"])
model.eval()

print("✅ Model loaded and ready.")


📒 Chord helper function

Convert chord names to 12-dimensional binary vectors.

In [ ]:
# Simple chord dictionary (expand as needed)
def roman_to_chord_vector(roman):
    chord_templates = {
        'C': [0, 4, 7],
        'F': [5, 9, 0],
        'G': [7, 11, 2],
        'Am': [9, 0, 4],
        'Dm': [2, 5, 9],
        'Em': [4, 7, 11]
    }
    vec = np.zeros(12)
    for p in chord_templates.get(roman, []):
        vec[p % 12] = 1
    return vec


📒 Create a custom chord progression and convert to tensor


In [ ]:
# 8 bars × 16 frames per bar = 128 time steps
progression = ['C', 'Am', 'F', 'G', 'C', 'Am', 'F', 'G']
frames_per_bar = model_config["frame_per_bar"]
max_len = model_config["num_bars"] * frames_per_bar

chord_matrix = np.zeros((max_len, 12))
for i, chord in enumerate(progression):
    vec = roman_to_chord_vector(chord)
    chord_matrix[i * frames_per_bar:(i + 1) * frames_per_bar] = vec

chord_tensor = torch.tensor(chord_matrix).unsqueeze(0).float()  # shape: [1, 128, 12]
print(f"✅ chord_tensor shape: {chord_tensor.shape}")



📒 Generate melody from scratch (no prime)


In [ ]:
# Empty prime (start from scratch)
prime_rhythm = torch.zeros((1, 0), dtype=torch.long)
prime_pitch = torch.zeros((1, 0), dtype=torch.long)

# Sampling
with torch.no_grad():
    result = model.sampling(prime_rhythm, prime_pitch, chord_tensor, topk=3)

pitch = result["pitch"][0].numpy()
rhythm = result["rhythm"][0].numpy()
print("✅ Melody generated.")


In [ ]:
output_path = "generated_custom.mid"
pitch_to_midi(pitch, chord_matrix, frame_per_bar=frames_per_bar, save_path=output_path)
print(f"✅ MIDI saved as {output_path}")


In [ ]:
import pretty_midi

midi_data = pretty_midi.PrettyMIDI(output_path)
melody_roll = midi_data.instruments[0].get_piano_roll(fs=16)

plt.figure(figsize=(12, 4))
plt.imshow(melody_roll, origin='lower', aspect='auto', cmap='gray_r')
plt.title("Generated Melody - Piano Roll")
plt.xlabel("Time")
plt.ylabel("MIDI Pitch")
plt.savefig("generated_custom_pianoroll.png")
print("✅ Piano roll saved as generated_custom_pianoroll.png")


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Lista de acordes disponibles
available_chords = ['C', 'F', 'G', 'Am', 'Dm', 'Em']

# Crear 8 menús desplegables (uno por compás)
chord_selectors = [widgets.Dropdown(options=available_chords, description=f'Bar {i+1}:') for i in range(8)]
generate_button = widgets.Button(description="Generate Chord Tensor", button_style='success')
output_area = widgets.Output()

def on_generate_clicked(b):
    with output_area:
        clear_output()
        # Leer progresión seleccionada
        progression = [dropdown.value for dropdown in chord_selectors]
        print("🎼 Selected progression:", progression)
        
        # Generar matriz de acordes
        frames_per_bar = model_config["frame_per_bar"]
        total_frames = frames_per_bar * len(progression)
        chord_matrix = np.zeros((total_frames, 12))
        for i, chord in enumerate(progression):
            vec = roman_to_chord_vector(chord)
            chord_matrix[i * frames_per_bar:(i + 1) * frames_per_bar] = vec

        # Convertir a tensor y asignar globalmente
        global chord_tensor
        chord_tensor = torch.tensor(chord_matrix).unsqueeze(0).float()
        print(f"✅ chord_tensor shape: {chord_tensor.shape} (ready to sample)")

generate_button.on_click(on_generate_clicked)

# Mostrar UI
display(widgets.VBox(chord_selectors + [generate_button, output_area]))


In [ ]:
result = model.sampling(prime_rhythm, prime_pitch, chord_tensor, topk=3)


In [ ]:
import sys
sys.path.append("../utils")  # where chord_library.py was saved

import ipywidgets as widgets
from IPython.display import display, clear_output
import chord_library as chords
import torch
import numpy as np

# All available Roman numeral chords from the module
available_chords = list(chords.ROMAN_TO_DEGREES.keys())

# Create 8 dropdowns
chord_selectors = [widgets.Dropdown(options=available_chords, value='I', description=f'Bar {i+1}') for i in range(8)]
generate_button = widgets.Button(description="Generate Chord Tensor", button_style='success')
output_area = widgets.Output()

def on_generate_clicked(b):
    with output_area:
        clear_output()
        selected_progression = [selector.value for selector in chord_selectors]
        print("🎼 Selected progression:", selected_progression)

        try:
            matrix = chords.chord_progression_to_matrix(selected_progression, frames_per_chord=16)
            global chord_tensor
            chord_tensor = torch.tensor(matrix).unsqueeze(0).float()
            print(f"✅ chord_tensor ready with shape: {chord_tensor.shape}")
        except Exception as e:
            print(f"❌ Error: {e}")

generate_button.on_click(on_generate_clicked)

# Display UI
display(widgets.VBox(chord_selectors + [generate_button, output_area]))


In [ ]:
with torch.no_grad():
    result = model.sampling(prime_rhythm, prime_pitch, chord_tensor, topk=3)


In [ ]:
pitch = result["pitch"][0].numpy()
rhythm = result["rhythm"][0].numpy()
print("✅ Melody generated.")

In [ ]:
output_path = "generated_custom2.mid"
pitch_to_midi(pitch, chord_matrix, frame_per_bar=frames_per_bar, save_path=output_path)
print(f"✅ MIDI saved as {output_path}")
